In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver # driver de selenium

import time   # para manejar tiempo

In [2]:
from selenium.webdriver.chrome.options import Options   # opciones del driver

from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...

from selenium.webdriver.support.ui import WebDriverWait   # es para esperar

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas

In [3]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

#### 1. Conecto con la página WEB

In [6]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                                # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

In [7]:
url = 'https://www.entradas.com/city/madrid-370/conciertos-y-festivales-85/'

driver.get(url)

Conecto con la Web de Entradas.com para trabajar sobre esta, y hacer scraping de los datos que quiero. Utilizaré el método de Scrapeo de selenium.

#### 2. Aceptar las cookies 

- Lo primero que hago es darle a Aceptas las cookies del sitio web. 
- Para ello tengo un problema inicial, ya que está categorizada como una pestaña "shadow" por lo que cuando busco la opción de hacer click al botón de aceptar, aparece como una pestaña / lista vacía. 

In [8]:
div_shadow = driver.find_element(By.XPATH, '//div[@class="cmpwrapper"]')

In [9]:
# Check if the element has a Shadow DOM using JavaScript
has_shadow_dom = driver.execute_script('return arguments[0].shadowRoot !== null', div_shadow)

if has_shadow_dom:
    print("Element has a Shadow DOM")
else:
    print("Element does not have a Shadow DOM")

Element has a Shadow DOM


In [10]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', div_shadow)

In [11]:
shadow_root.find_elements(By.CSS_SELECTOR, '#cmpwelcomebtnyes > a')[0].click()

- Una vez consigo traspasar la barrera del 'Shadow DOM', entonces puedo comenzar con el scrapeo de las tablas que necesito para mi estudio. 

#### 3. Empezamos el scrapeo por eventos en tabla

In [12]:
tabla = '//product-group-item'

In [13]:
table = driver.find_elements(By.XPATH, tabla)
table

[<selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_30")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_32")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_33")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_34")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_35")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804

In [14]:
type(table)

list

In [15]:
elementos = driver.find_elements(By.XPATH, '//*[@id="list-item-0"]')
elementos

[<selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_50")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_51")>]

In [16]:
type(elementos)

list

In [18]:
elementos = elementos[0].text.split('\n')
elementos

['Cariño',
 'Inverfest 2024',
 '13/01/20241 Evento',
 ' desde 19,80 €',
 'Mostrar 1 Evento']

In [66]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_52")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="73E31C7884D5290B1A8D781383B8B66E_element_53")>]

In [104]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//*[@class = "btn theme-button-radius btn-sm btn-block btn-primary theme-interaction-btn-bg theme-btn-font-color theme-btn-font-color-hover"]')
    enlaces = [enl.get_attribute('href') for enl in enlace]

    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Cariño', 'Iván Ferreiro', 'The World Of Hans Zimmer', 'Triángulo de Amor Bizarro', 'Nico B', 'Cala Vento', 'Sidonie', 'Belako - Sigo regando', 'Biznaga', 'Morreo - Alegría Club!', 'Diego el Cigala - Obras Maestras', 'Symphonic Rhapsody of Queen', 'Sexy Zebras', 'Biografía - Rafa Sánchez, de la unión', 'OBK', 'El Consorcio - Toda una vida', 'Rock en Familia - Hijos del Rock and Roll', 'Rock en Familia - I Love Rock and Roll', 'Rock en Familia - Queenmanía', 'Rock en Familia - I Love Heavy Metal'], ['13/01/2024', '1 Evento', '13/01/2024', '1 Evento', '24/04/2024 ─ 29/04/2024', '2 eventos', '29/12/2023', '1 Evento', '14/01/2024', '1 Evento', '01/12/2023', '1 Evento', '12/01/2024 ─ 14/01/2024', '2 eventos', '20/01/2024', '1 Evento', '26/01/2024', '1 Evento', '18/01/2024', '1 Evento', '11/12/2023 ─ 18/12/2023', '2 eventos', '16/01/2024', '1 Evento', '01/02/2024', '1 Evento', '11/12/2023', '1 Evento', '13/01/2024 ─ 27/01/2024', '3 eventos', '27/11/2023 ─ 28/11/2023', '2 eventos', '05/11/2

In [105]:
type(entry)

list

In [106]:
artistas = entry[0]
artistas

['Cariño',
 'Iván Ferreiro',
 'The World Of Hans Zimmer',
 'Triángulo de Amor Bizarro',
 'Nico B',
 'Cala Vento',
 'Sidonie',
 'Belako - Sigo regando',
 'Biznaga',
 'Morreo - Alegría Club!',
 'Diego el Cigala - Obras Maestras',
 'Symphonic Rhapsody of Queen',
 'Sexy Zebras',
 'Biografía - Rafa Sánchez, de la unión',
 'OBK',
 'El Consorcio - Toda una vida',
 'Rock en Familia - Hijos del Rock and Roll',
 'Rock en Familia - I Love Rock and Roll',
 'Rock en Familia - Queenmanía',
 'Rock en Familia - I Love Heavy Metal']

In [107]:
fecha_eventos = entry[1]
fecha_eventos

['13/01/2024',
 '1 Evento',
 '13/01/2024',
 '1 Evento',
 '24/04/2024 ─ 29/04/2024',
 '2 eventos',
 '29/12/2023',
 '1 Evento',
 '14/01/2024',
 '1 Evento',
 '01/12/2023',
 '1 Evento',
 '12/01/2024 ─ 14/01/2024',
 '2 eventos',
 '20/01/2024',
 '1 Evento',
 '26/01/2024',
 '1 Evento',
 '18/01/2024',
 '1 Evento',
 '11/12/2023 ─ 18/12/2023',
 '2 eventos',
 '16/01/2024',
 '1 Evento',
 '01/02/2024',
 '1 Evento',
 '11/12/2023',
 '1 Evento',
 '13/01/2024 ─ 27/01/2024',
 '3 eventos',
 '27/11/2023 ─ 28/11/2023',
 '2 eventos',
 '05/11/2023 ─ 10/03/2024',
 '2 eventos',
 '03/12/2023',
 '1 Evento',
 '17/12/2023 ─ 14/04/2024',
 '2 eventos',
 '19/11/2023 ─ 24/03/2024',
 '2 eventos']

In [108]:
precios = entry[2]
precios


['desde 19,80 €',
 'desde 38,50 €',
 'desde 55,00 €',
 'desde 19,80 €',
 'desde 13,50 €',
 'desde 19,80 €',
 'desde 27,50 €',
 'desde 17,60 €',
 'desde 18,70 €',
 'desde 14,50 €',
 'desde 55,00 €',
 'desde 35,00 €',
 'desde 18,00 €',
 'desde 35,00 €',
 'desde 30,00 €',
 'desde 29,00 €',
 'desde 7,50 €',
 'desde 7,50 €',
 'desde 7,50 €',
 'desde 7,50 €']

In [109]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024', '13/01/2024', '24/04/2024 ─ 29/04/2024', '29/12/2023', '14/01/2024', '01/12/2023', '12/01/2024 ─ 14/01/2024', '20/01/2024', '26/01/2024', '18/01/2024', '11/12/2023 ─ 18/12/2023', '16/01/2024', '01/02/2024', '11/12/2023', '13/01/2024 ─ 27/01/2024', '27/11/2023 ─ 28/11/2023', '05/11/2023 ─ 10/03/2024', '03/12/2023', '17/12/2023 ─ 14/04/2024', '19/11/2023 ─ 24/03/2024']
['1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '3 eventos', '2 eventos', '2 eventos', '1 Evento', '2 eventos', '2 eventos']


In [110]:
import pandas as pd

In [111]:
datos = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

datos

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €",https://www.entradas.com/artist/sidonie/sidoni...
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/belako/
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €",https://www.entradas.com/artist/biznaga/
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €",https://www.entradas.com/artist/morreo/


In [114]:
base_url = 'https://www.entradas.com/city/madrid-370/conciertos-y-festivales-85/?page='

# Número de páginas a recorrer
num_paginas = 5

for pagina in range(1, num_paginas + 1):
    url = base_url + str(pagina)
    driver.get(url)

In [116]:
datos

,artista,fechas,eventos,precios,enlaces
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/carino/carino-...
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €",https://www.entradas.com/artist/ivan-ferreiro/
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €",https://www.entradas.com/artist/hans-zimmer/
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/triangulo-de-a...
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €",https://www.entradas.com/artist/nico-b/
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €",https://www.entradas.com/artist/cala-vento/
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €",https://www.entradas.com/artist/sidonie/sidoni...
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €",https://www.entradas.com/artist/belako/
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €",https://www.entradas.com/artist/biznaga/
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €",https://www.entradas.com/artist/morreo/


In [117]:
table = driver.find_elements(By.XPATH, tabla)
table

[<selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12307")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12308")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12309")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12310")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12311")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12312")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73

In [118]:
len(table)

62

In [120]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12371")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3c0e53c73e77f490db35a5d4804eb44", element="D0D199E13AE128AB7AC86C1A29D48963_element_12372")>]

In [129]:
elements[1].text

'Rodrigo y Gabriela - In Between Thoughts...A New World Tour 2023\n07/11/2023 ─ 09/11/20232 eventos\n  desde 30,00 €\nMostrar los 2 eventos\nMadrid, mar. 07/11/2023, 21:00\nRodrigo y Gabriela - In Between Thoughts...A New World Tour 2023\n  Entradas\nHard GZ: Tour 23/24\n22/02/20241 Evento\n desde 20,00 €\nMostrar 1 Evento\nMadrid, jue. 22/02/2024, 21:00\nHard GZ: Tour 23/24\n Entradas\nAlice Phoebe Lou\nCiclo Jaguar\n29/11/20231 Evento\n  desde 24,00 €\nMostrar 1 Evento\nMadrid, mié. 29/11/2023, 20:00\nAlice Phoebe Lou\n  Entradas\nEl Conciertazo en Familia: Especial Navidad\nConcierto guionizado en el que descubrir que la Música Clásica también puede ser divertida.\n30/12/20231 Evento\n desde 18,00 €\nMostrar 1 Evento\nMadrid, sáb. 30/12/2023, 12:00\nEl Conciertazo en Familia: Especial Navidad\n Entradas\nSchumann y Brahms - Ensemble Praeteritum\n16/11/20231 Evento\n desde 12,00 €\nMostrar 1 Evento\nMadrid, jue. 16/11/2023, 20:00\nSchumann y Brahms - Ensemble Praeteritum\n Entradas\n

In [131]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    enlace = element.find_elements(By.XPATH, '//*[@class = "btn theme-button-radius btn-sm btn-block btn-primary theme-interaction-btn-bg theme-btn-font-color theme-btn-font-color-hover"]')
    enlaces = [enl.get_attribute('href') for enl in enlace]

    data.append([astists, dates, precios, enlaces])

for entry in data:
    print(entry)

[['Rodrigo y Gabriela - In Between Thoughts...A New World Tour 2023', 'Hard GZ: Tour 23/24', 'Alice Phoebe Lou', 'El Conciertazo en Familia: Especial Navidad', 'Schumann y Brahms - Ensemble Praeteritum', 'Jaleos Jondos', 'Aranlight Soul & Gospel - Jazz & the City', 'Homenaje a Pablo Milanés', 'Fred de Palma', 'Momentos Alhambra: Rikas', 'Lo + actual', 'Mujer, inspiración y acción', 'Rolling Stones vs Beatles', 'La Serva Padrona - Aranjuez', 'Kaze - Gira 2024', 'Luceral + Cherry Pie', 'The Greatest Love Of All', 'Danza para Guitarra', 'Blues & Roots', 'Invisible Tattoo', 'Rupatrupa', 'Footloosing', 'Luna Ki', 'Diego Guerrero', 'Rhythm Of The Dance', 'The Music of Big Star + The Sadies', 'Eva Català Group', 'Aires de Persia', 'Impulso presenta: Danilo Plessow (Motor City Drum Ensemble) + Javi Frias & Roberto Tempo', 'Fran Perea "1 más 1 son 20"', 'Rawayana - ¿Quien trae las cornetas?', 'Apartamentos Acapulco + Cora Yako', 'Gran Gala Flamenco - Madrid', 'VibranDo', 'The Other Side - A Pin

In [132]:
datos_2 = pd.DataFrame({'artista': artistas, 'fechas': fechas, 'eventos': num_eventos, 'precios': precios, 'enlaces': enlaces})

datos_2

ValueError: All arrays must be of the same length

In [126]:
len(fechas)

20